In [ ]:
# !unzip -o /content/dataset.zip -d /content/

In [ ]:
# !rm -rf /content/dataset

In [ ]:
'''
import os
import cv2

def convert_images_to_jpeg(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            # Cargar la imagen
            img = cv2.imread(file_path)

            if img is not None:
                # Guardar la imagen en formato JPEG
                new_file_path = file_path.split('.')[0] + '.jpg'
                cv2.imwrite(new_file_path, img)
                os.remove(file_path)  # Eliminar la imagen original
                print(f"Convertida {file_path} a {new_file_path}")

# Convertir las imágenes en ambas carpetas
convert_images_to_jpeg('/content/dataset/validas')
convert_images_to_jpeg('/content/dataset/no_validas')
'''

In [ ]:
'''
import os

validas_dir = '/content/dataset/validas'
no_validas_dir = '/content/dataset/no_validas'

# Contar las imágenes en cada carpeta
validas_count = len([f for f in os.listdir(validas_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
no_validas_count = len([f for f in os.listdir(no_validas_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])

print(f"Imágenes válidas: {validas_count}")
print(f"Imágenes no válidas: {no_validas_count}")

validas_files = [f for f in os.listdir(validas_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
print(validas_files)
'''

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

# Define the paths to your dataset
base_dir = '/content/dataset'  # Cambia esta ruta con el directorio de tu dataset

# Directorios para imágenes válidas y no válidas
validas_dir = os.path.join(base_dir, 'validas')
no_validas_dir = os.path.join(base_dir, 'no_validas')

# Preprocesamiento de imágenes y carga de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
)

# Cambiar las etiquetas para que 'validas' sea 0 y 'no_validas' sea 1
train_generator.class_indices = {'validas': 0, 'no_validas': 1}



In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria (válida o no válida)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Número de pasos por cada época
    epochs=10,            # Número de épocas
    verbose=1
)


In [ ]:
model.save('modelo_carnet.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Cargar el modelo
model = tf.keras.models.load_model('modelo_carnet.h5')

# Función para predecir si una imagen es válida
def predecir_imagen(imagen_path):
    img = image.load_img(imagen_path, target_size=(150, 150))  # Ajustar tamaño
    img_array = image.img_to_array(img)  # Convertir la imagen a array
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión extra
    img_array = img_array / 255.0  # Normalizar

    prediction = model.predict(img_array)  # Predicción
    if prediction[0] > 0.5:
        return "Válida"
    else:
        return "No válida"

# Usar la función
resultado = predecir_imagen('/content/prueba5.jpg')  # Cambia la ruta de la imagen
print(resultado)
